# k分割交差検証

ホールドアウト法よりもさらに頑健な性能評価方法。  
非復元抽出を用いて、訓練データセットをランダムにk個に分割する。そのうちのk-1個はいわゆる訓練フォールドであり、モデルの訓練に使う。残りの1個はいわゆるテストフォールドであり、性能の評価に使う。この手順をk回繰り返すことで、k個のモデルを取得し、モデルの平均性能を推定する。  
一般にモデルのチューニングに使う。  
Ron Kohavi：：バイアスとバリアンスのバランスが最もよいのは、k=10の交差検証？

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
import numpy as np

# データセットの読み込み
data = load_breast_cancer()

# 特徴量
X = data.data          # NumPy配列（shape: [n_samples, n_features]）
# ラベル（0: 悪性, 1: 良性）
y = data.target # LabelEncoder使う必要なし

from sklearn.model_selection import train_test_split
# データセットを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [4]:
import numpy as np
from sklearn.model_selection import StratifiedKFold # 層化KFold。クラスの比率を保ちながら分割。
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(
    StandardScaler(),
    PCA(n_components=2),
    LogisticRegression(random_state=1)
)

kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
scores = []
# イテレータのインデックスと要素をループ処理：（上から順に）
#     データをモデルに適合
#     テストデータの正解率を算出
#     正解率をscoresに追加
#     分割の番号、0以上の要素数、および正解率を表示
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print(f'Fold: {k+1:02d},'
          f'Class distr.: {np.bincount(y_train[train])},'
          f'Acc.: {score:.3f}')

# 平均と標準偏差を計算
mean_acc = np.mean(scores)
std_acc = np.std(scores)
print(f'\nCV accuracy: {mean_acc:.3f} (+/- {std_acc:.3f})')

Fold: 01,Class distr.: [153 256],Acc.: 0.978
Fold: 02,Class distr.: [153 256],Acc.: 0.935
Fold: 03,Class distr.: [153 256],Acc.: 0.957
Fold: 04,Class distr.: [153 256],Acc.: 0.935
Fold: 05,Class distr.: [153 256],Acc.: 0.913
Fold: 06,Class distr.: [153 257],Acc.: 0.956
Fold: 07,Class distr.: [153 257],Acc.: 0.933
Fold: 08,Class distr.: [153 257],Acc.: 0.956
Fold: 09,Class distr.: [153 257],Acc.: 0.933
Fold: 10,Class distr.: [153 257],Acc.: 0.978

CV accuracy: 0.947 (+/- 0.020)


In [5]:
# 関数
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)  # n_jobs=-1で全てのCPUコアを使用

print(f'Cross-validation scores: {scores}')  
print(f'CV accuracy: {np.mean(scores):.3f} (+/- {np.std(scores):.3f})')

Cross-validation scores: [0.97826087 0.93478261 0.95652174 0.93478261 0.91304348 0.95555556
 0.93333333 0.95555556 0.93333333 0.97777778]
CV accuracy: 0.947 (+/- 0.020)


cross_val_score関数のアプローチのきわめて便利な特徴の1つとして、フォールドごとの評価の計算処理をマシン上の複数のCPUに分散させることができる。n_jobs=-1でマシン上で利用可能な全てのCPUを使える。